# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,746.626540,0.807439,1234,747.433979,745.819101,1494.867958,1491.638201
6,996.584098,0.275450,1234,996.859548,996.308648,1993.719096,1992.617296
7,896.461466,-0.924642,1234,895.536824,897.386108,1791.073648,1794.772215
8,151.060298,-0.032586,1234,151.027712,151.092884,302.055424,302.185767
9,327.440197,0.532949,1234,327.973146,326.907248,655.946291,653.814497
10,832.072232,0.496936,1234,832.569169,831.575296,1665.138338,1663.150592
11,23.310940,-0.422701,1234,22.888239,23.733642,45.776478,47.467284
12,699.469059,0.100298,1234,699.569357,699.368762,1399.138714,1398.737523
13,505.801239,-0.083110,1234,505.718129,505.884349,1011.436258,1011.768698
14,152.947358,-0.380928,1234,152.566430,153.328286,305.132860,306.656573


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-747.433979,745.819101
6,-996.859548,996.308648
7,-895.536824,897.386108
8,-151.027712,151.092884
9,-327.973146,326.907248
10,-832.569169,831.575296
11,-22.888239,23.733642
12,-699.569357,699.368762
13,-505.718129,505.884349
14,-152.566430,153.328286


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-747.433979,745.819101
6,-996.859548,996.308648
7,-895.536824,897.386108
8,-151.027712,151.092884
9,-327.973146,326.907248
10,-832.569169,831.575296
11,-22.888239,23.733642
12,-699.569357,699.368762
13,-505.718129,505.884349
14,-152.566430,153.328286


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-747.43397914,  745.81910052],
       [-996.85954808,  996.30864782],
       [-895.53682381,  897.38610751],
       [-151.02771183,  151.09288357],
       [-327.97314561,  326.90724839],
       [-832.56916876,  831.57529591],
       [ -22.88823902,   23.73364197],
       [-699.56935697,  699.36876161],
       [-505.71812892,  505.88434881],
       [-152.56642986,  153.32828626]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1492.867958,-1491.638201
6,1991.719096,-1992.617296
7,1789.073648,-1794.772215
8,300.055424,-302.185767
9,653.946291,-653.814497
10,1663.138338,-1663.150592
11,43.776478,-47.467284
12,1397.138714,-1398.737523
13,1009.436258,-1011.768698
14,303.132860,-306.656573


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
